In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class DetectSmileModel:
    def __init__(self, data_dir, img_size=(100, 100), batch_size=30):
        self.initial_epochs = 10
        self.data_dir = data_dir
        self.img_size = img_size
        self.batch_size = batch_size
        self.history = None
        self.test_dataset = None
        self.validation_dataset = None
        self.train_dataset = None
        self.model = None
        self.class_names = None

    def preprocess_input(self, x):
        return tf.keras.applications.vgg16.preprocess_input(x)

    def build_model(self):
        # Load the training dataset
        self.train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
            self.data_dir,
            validation_split=0.2,
            subset="training",
            seed=123,
            image_size=self.img_size,
            batch_size=self.batch_size
        )

        self.class_names = self.train_dataset.class_names
        print(self.class_names)

        # Load the validation dataset
        self.validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
            self.data_dir,
            validation_split=0.2,
            subset="validation",
            seed=123,
            image_size=self.img_size,
            batch_size=self.batch_size
        )

        # Create the test dataset
        val_batches = tf.data.experimental.cardinality(self.validation_dataset)
        self.test_dataset = self.validation_dataset.take(val_batches // 5)
        validation_dataset = self.validation_dataset.skip(val_batches // 5)

        # Improve performance using prefetching
        AUTOTUNE = tf.data.AUTOTUNE

        self.train_dataset = self.train_dataset.prefetch(buffer_size=AUTOTUNE)
        self.validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
        self.test_dataset = self.test_dataset.prefetch(buffer_size=AUTOTUNE)

        # Load the base VGG16 model
        base_model = tf.keras.applications.vgg16.VGG16(input_shape=self.img_size + (3,), include_top=False,
                                                       weights="imagenet")

        # Unfreeze the base model layers
        base_model.trainable = True

        # Build the model architecture
        inputs = tf.keras.Input(shape=self.img_size + (3,))
        x = self.preprocess_input(inputs)
        x = base_model(x, training=True)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

        self.model = tf.keras.Model(inputs, outputs)

        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        self.model.summary()

        # Evaluate the model on the validation dataset
        loss0, accuracy0 = self.model.evaluate(validation_dataset)
        print("Initial loss: {:.2f}".format(loss0))
        print("Initial accuracy: {:.2f}".format(accuracy0))

        # Train the model
        self.history = self.model.fit(
            self.train_dataset,
            epochs=self.initial_epochs,
            validation_data=validation_dataset,
            validation_steps=len(validation_dataset),
            steps_per_epoch=len(self.train_dataset),
        )

        # Enable fine-tuning
        base_model.trainable = True

        # Fine-tune from a specific layer onwards
        fine_tune_at = 10

        # Freeze layers before the fine-tuning layer
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False

        # Recompile the model for fine-tuning
        self.model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                           optimizer='adam',
                           metrics=['accuracy'])

        self.model.summary()

    def fine_tune(self, epochs):
        total_epochs = self.initial_epochs + epochs

        # Fine-tune the model
        history_fine = self.model.fit(self.train_dataset,
                                      epochs=total_epochs,
                                      initial_epoch=self.history.epoch[-1],
                                      validation_data=self.validation_dataset)
        return history_fine


    def report(self):
        acc = self.history.history['accuracy']
        val_acc = self.history.history['val_accuracy']

        loss = self.history.history['loss']
        val_loss = self.history.history['val_loss']

        return acc, val_acc, loss, val_loss

    def evaluate(self):
        loss, accuracy = self.model.evaluate(self.test_dataset)
        print('Test accuracy :', accuracy)

    def save_model(self, save_path):
        if self.model is not None:
            self.model.save(save_path)
            print("Model saved successfully.")
        else:
            print("No model to save.")

    def load_model(self, model_path):
        self.model = tf.keras.models.load_model(model_path)
        print("Model loaded successfully.")

    def predict(self, image):
        if self.model is not None:
            image = tf.expand_dims(image, 0)
            image = self.preprocess_input(image)
            prediction = self.model.predict(image)[0][0]
            if prediction < 0.5:
                return "No Smile"
            else:
                return "Smile"
        else:
            print("No model loaded.")

    def prediction(self):
        # Retrieve a batch of images from the test set
        image_batch, label_batch = self.test_dataset.as_numpy_iterator().next()
        predictions = self.model.predict_on_batch(image_batch).flatten()

        # Apply a sigmoid since our model returns logits
        predictions = tf.nn.sigmoid(predictions)
        predictions = tf.where(predictions < 0.5, 0, 1)

        print('Predictions:\n', predictions.numpy())
        print('Labels:\n', label_batch)

In [ ]:
# Example usage:
model = DetectSmileModel(data_dir='drive/MyDrive/ColabDrive/SmileDetection/')
model.build_model()
model.save_model('vgg_model.h5')

Found 5891 files belonging to 2 classes.
Using 4713 files for training.
['not_smile', 'smile']
Found 5891 files belonging to 2 classes.
Using 1178 files for validation.
58889256/58889256 [==============================] - 2s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 tf.__operators__.getitem (  (None, 100, 100, 3)       0         
 SlicingOpLambda)                                                
                                                                 
 tf.nn.bias_add (TFOpLambda  (None, 100, 100, 3)       0         
 )                                                               
                                                                 
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                         

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.evaluate()

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5820: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


8/8 [==============================] - 3s 68ms/step - loss: 0.2671 - accuracy: 0.9125
Test accuracy : 0.9125000238418579


In [ ]:
model.fine_tune(10)

Epoch 10/20
158/158 [==============================] - 11s 68ms/step - loss: 0.1735 - accuracy: 0.9414 - val_loss: 0.2561 - val_accuracy: 0.9168
Epoch 11/20
158/158 [==============================] - 12s 74ms/step - loss: 0.1699 - accuracy: 0.9417 - val_loss: 0.2201 - val_accuracy: 0.9211
Epoch 12/20
158/158 [==============================] - 13s 79ms/step - loss: 0.1592 - accuracy: 0.9457 - val_loss: 0.2275 - val_accuracy: 0.9179
Epoch 13/20
158/158 [==============================] - 12s 75ms/step - loss: 0.1613 - accuracy: 0.9438 - val_loss: 0.2740 - val_accuracy: 0.9190
Epoch 14/20
158/158 [==============================] - 12s 72ms/step - loss: 0.1778 - accuracy: 0.9438 - val_loss: 0.3645 - val_accuracy: 0.9158
Epoch 15/20
158/158 [==============================] - 13s 79ms/step - loss: 0.1823 - accuracy: 0.9414 - val_loss: 0.2524 - val_accuracy: 0.9211
Epoch 16/20
158/158 [==============================] - 12s 74ms/step - loss: 0.1890 - accuracy: 0.9389 - val_loss: 0.2356 - val_ac

In [ ]:
model.save_model("vgg_model_ft_one.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved successfully.


In [ ]:
model.evaluate()

8/8 [==============================] - 1s 40ms/step - loss: 0.2951 - accuracy: 0.9292
Test accuracy : 0.9291666746139526


In [ ]:
model.fine_tune(10)

Epoch 10/20
158/158 [==============================] - 12s 74ms/step - loss: 0.1424 - accuracy: 0.9542 - val_loss: 0.2463 - val_accuracy: 0.9254
Epoch 11/20
158/158 [==============================] - 15s 94ms/step - loss: 0.1476 - accuracy: 0.9489 - val_loss: 0.2822 - val_accuracy: 0.9222
Epoch 12/20
158/158 [==============================] - 11s 67ms/step - loss: 0.1353 - accuracy: 0.9546 - val_loss: 0.2889 - val_accuracy: 0.9243
Epoch 13/20
158/158 [==============================] - 12s 73ms/step - loss: 0.1265 - accuracy: 0.9567 - val_loss: 0.2702 - val_accuracy: 0.9211
Epoch 14/20
158/158 [==============================] - 15s 91ms/step - loss: 0.1199 - accuracy: 0.9563 - val_loss: 0.2383 - val_accuracy: 0.9286
Epoch 15/20
158/158 [==============================] - 11s 69ms/step - loss: 0.1108 - accuracy: 0.9593 - val_loss: 0.3415 - val_accuracy: 0.9264
Epoch 16/20
158/158 [==============================] - 12s 77ms/step - loss: 0.1083 - accuracy: 0.9597 - val_loss: 0.2986 - val_ac

In [ ]:
model.save_model("vgg_model_ft_two.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved successfully.


In [ ]:
model.evaluate()

8/8 [==============================] - 1s 40ms/step - loss: 0.3163 - accuracy: 0.9208
Test accuracy : 0.9208333492279053


In [ ]:
model.fine_tune(5)

Epoch 10/15
158/158 [==============================] - 12s 72ms/step - loss: 0.1155 - accuracy: 0.9620 - val_loss: 0.3036 - val_accuracy: 0.9232
Epoch 11/15
158/158 [==============================] - 12s 70ms/step - loss: 0.0919 - accuracy: 0.9684 - val_loss: 0.3261 - val_accuracy: 0.9158
Epoch 12/15
158/158 [==============================] - 12s 73ms/step - loss: 0.0789 - accuracy: 0.9716 - val_loss: 0.3170 - val_accuracy: 0.9200
Epoch 13/15
158/158 [==============================] - 14s 84ms/step - loss: 0.0834 - accuracy: 0.9680 - val_loss: 0.2903 - val_accuracy: 0.9275
Epoch 14/15
158/158 [==============================] - 12s 72ms/step - loss: 0.0737 - accuracy: 0.9722 - val_loss: 0.3342 - val_accuracy: 0.9264
Epoch 15/15
158/158 [==============================] - 14s 84ms/step - loss: 0.0788 - accuracy: 0.9720 - val_loss: 0.3596 - val_accuracy: 0.9232


In [ ]:
model.save_model("vgg_model_ft_three.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved successfully.


In [ ]:
model.evaluate()

8/8 [==============================] - 1s 59ms/step - loss: 0.4047 - accuracy: 0.9125
Test accuracy : 0.9125000238418579
